In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = stopwords.words('english')
import gensim
from gensim.models import Word2Vec, KeyedVectors
import gc
import ast
from string import punctuation
import string
gc.collect()
import spacy
from textblob import TextBlob
import emoji
from textattack.augmentation import EasyDataAugmenter
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import kagglehub
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from transformers import BertTokenizer, BertModel
from transformers import RobertaTokenizerFast, TFRobertaModel
from transformers import TFXLNetModel, XLNetTokenizer
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [49]:
path = kagglehub.dataset_download('abhi8923shriv/sentiment-analysis-dataset')
train_dataset = path+'/train.csv'
test_dataset = path+'/test.csv'
train_df = pd.read_csv(train_dataset, encoding='ISO-8859-1')
test_df = pd.read_csv(test_dataset, encoding='ISO-8859-1')

In [50]:
def abbreviation_replacement(text):
  text = re.sub(r"i\'m", "i am", text)
  text = re.sub(r"\'re", "are", text)
  text = re.sub(r"he\'s", "he is", text)
  text = re.sub(r"it\'s", "it is", text)
  text = re.sub(r"that\'s", "that is", text)
  text = re.sub(r"who\'s", "who is", text)
  text = re.sub(r"what\'s", "what is", text)
  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r",", " , ", text)
  text = re.sub(r"!", " ! ", text)
  text = re.sub(r"\.", " \. ", text)
  text = re.sub(r"\(", " \( ", text)
  text = re.sub(r"\)", " \) ", text)
  text = re.sub(r"\?", " \? ", text)
  return text

# handle emojis, source: https://github.com/Carmezim/crypto-twitter-sentiment-analysis/blob/master/preprocess.py
def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' pos_emo ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' pos_emo ', tweet)
    # Love -- <3, :*, ♥, ^^, ^_^
    tweet = re.sub(r'(<3|:\*|♥|\^\^|\^_\^)', ' pos_emo ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' pos_emo ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' neg_emo ', tweet)
    # Cry -- :,(, :'(, :"(, ;(, );
    tweet = re.sub(r'(:,\(|:\'\(|:"\(|;\(|\(\;)', ' neg_emo ', tweet)
    return tweet

puntation_dict = str.maketrans('', '', string.punctuation)
def drop_punctuation(text):
  text = re.sub(r'(.)\1+', r'\1\1', text)
  text = text.translate(puntation_dict)
  return text

def remove_number(text):
  text = re.sub(r'(0|1|2|3|4|5|6|7|8|9)', '', text)
  return text

stoplists = [stoplist for stoplist in stopwords.words('english') if len(stoplist) <= 1]
nlp = spacy.load("en_core_web_sm")
def spacy_fast_lemmatizier_stopworder(text, return_tokens=False):
  doc = nlp(text)
  tokens = [token.lemma_ for token in doc if token.text not in stoplists]

  if return_tokens:
      return tokens

  text = ' '.join(tokens)
  return text

def preprocessing_all_method(tweet):
  tweet = tweet.strip().lower()
  tweet = abbreviation_replacement(tweet)
  tweet = handle_emojis(tweet)
  tweet = drop_punctuation(tweet)
  tweet = remove_number(tweet)
  tweet = spacy_fast_lemmatizier_stopworder(tweet, return_tokens=False)
  return tweet

def preprocess(texts):
  for i in tqdm(range(len(texts))):
      texts[i] = preprocessing_all_method(texts[i])
  return texts

In [51]:
def get_text_length(data):
  data["text_length"] = data["text"].str.len()
  data["num_words"] = data["text"].str.split(" ").str.len()
  return data

def get_num_profanity(text):
  with open("en.txt", 'r') as file:
    lines = file.readlines()
    profanity_list = [line.strip() for line in lines]
  count = 0
  for word in text.split():
    if word in profanity_list:
      count += 1
  return count

def get_capital_percentage(text):
  capital_letters = sum(1 for char in text if 'A' <= char <= 'Z')
  total_letters = sum(1 for char in text if char.isalpha())
  if total_letters == 0:
     return 0
  return capital_letters / total_letters

def get_polarity_subjectivity(text):
  blob = TextBlob(text)
  return blob.sentiment.polarity, blob.sentiment.subjectivity

def get_pos_counts(text):
  doc = nlp(text)
  pos_counts = {
      'num_nouns': 0,
      'num_verbs': 0,
      'num_adjs': 0,
      'num_advs': 0
  }
  for token in doc:
      if token.pos_ == 'NOUN':
          pos_counts['num_nouns'] += 1
      elif token.pos_ == 'VERB':
          pos_counts['num_verbs'] += 1
      elif token.pos_ == 'ADJ':
          pos_counts['num_adjs'] += 1
      elif token.pos_ == 'ADV':
          pos_counts['num_advs'] += 1
  return pos_counts

def count_punctuations(text):
  return text.count('!'), text.count('?'), text.count('...')

def count_uppercase_words(text):
  return sum(1 for word in text.split() if word.isupper())

def count_negations(text):
  negation_words = ['no', 'not', 'never', "n't"]
  return sum(1 for word in text.lower().split() if word in negation_words)

def count_emojis(text):
  return emoji.emoji_count(text)

def extract_features(df):
  df = get_text_length(df)
  features = {
      "num_profanity" : [],
      "capital_percentage" : [],
      'polarity' : [],
      'subjectivity' : [],
      'num_nouns' : [],
      'num_verbs' : [],
      'num_adjs' : [],
      'num_advs' : [],
      'num_exclamations' : [],
      'num_questions' : [],
      'num_ellipsis' : [],
      'uppercase_count' : [],
      'negation_count' : [],
      'emoji_count' : []
  }
  for text in tqdm(df['text']):
    features["num_profanity"  ].append(get_num_profanity(text))
    features["capital_percentage"].append(get_capital_percentage(text))
    polarity, subjectivity = get_polarity_subjectivity(text)
    features['polarity'].append(polarity)
    features['subjectivity'].append(subjectivity)
    pos_counts = get_pos_counts(text)
    features['num_nouns'].append(pos_counts['num_nouns'])
    features['num_verbs'].append(pos_counts['num_verbs'])
    features['num_adjs'].append(pos_counts['num_adjs'])
    features['num_advs'].append(pos_counts['num_advs'])
    exclamations, questions, ellipsis = count_punctuations(text)
    features['num_exclamations'].append(exclamations)
    features['num_questions'].append(questions)
    features['num_ellipsis'].append(ellipsis)
    uppercase_count = count_uppercase_words(text)
    features['uppercase_count'].append(uppercase_count)
    negation_count = count_negations(text)
    features['negation_count'].append(negation_count)
    emoji_count = count_emojis(text)
    features['emoji_count'].append(emoji_count)

  feature_df = pd.DataFrame(features)

  return feature_df

In [144]:
features_train = extract_features(train_df.fillna(""))
train_df["cleaned_text"] = preprocess(train_df["text"].fillna(""))
train_df["text_length"] = train_df["text"].str.len()
train_df["num_words"] = train_df["text"].str.split(" ").str.len()
train_merged = pd.concat([train_df[["cleaned_text", "text_length", "num_words", "sentiment"]], features_train], axis = 1)

features_test = extract_features(test_df.fillna(""))
test_df["cleaned_text"] = preprocess(test_df["text"].fillna(""))
test_df["text_length"] = test_df["text"].str.len()
test_df["num_words"] = test_df["text"].str.split(" ").str.len()
test_merged = pd.concat([test_df[["cleaned_text", "text_length", "num_words", "sentiment"]], features_test], axis = 1)

100%|██████████| 4815/4815 [00:23<00:00, 208.49it/s]


In [145]:
train_merged.to_csv("train_merged.csv", index = False)
test_merged.to_csv("test_merged.csv", index = False)

In [5]:
train_merged = pd.read_csv("train_merged.csv")
train_merged["cleaned_text"] = train_merged["cleaned_text"].fillna("")
train_merged = train_merged.fillna(0)
test_merged = pd.read_csv("test_merged.csv").dropna()

In [78]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_merged["sentiment"].values)
x = train_merged.drop("sentiment", axis = 1)
y_test = label_encoder.transform(test_merged["sentiment"].values)
x_test = test_merged.drop("sentiment", axis = 1)

In [79]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, stratify=y, random_state=42, test_size=0.1, shuffle=True)

# Word2Vec

In [156]:
tokenizer = Tokenizer(num_words=100000, oov_token="<OOV>")
tokenizer.fit_on_texts(x_train["cleaned_text"])
x_train_seq = tokenizer.texts_to_sequences(x_train["cleaned_text"])
x_valid_seq = tokenizer.texts_to_sequences(x_valid["cleaned_text"])
x_test_seq = tokenizer.texts_to_sequences(x_test["cleaned_text"])
max_length = 200
x_train_pad = pad_sequences(x_train_seq, maxlen=max_length, padding='post')
x_valid_pad = pad_sequences(x_valid_seq, maxlen=max_length, padding='post')
x_test_pad = pad_sequences(x_test_seq, maxlen=max_length, padding='post')

In [71]:
word2vec = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

embedding_dim = 300
word_index = tokenizer.word_index
num_words = min(100000, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i >= num_words:
        continue
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]

In [11]:
x_train_num = x_train.drop("cleaned_text", axis = 1).values
x_valid_num = x_valid.drop("cleaned_text", axis = 1).values
x_test_num = x_test.drop("cleaned_text", axis = 1).values
y_train_cat = to_categorical(y_train, num_classes=3)
y_valid_cat = to_categorical(y_valid, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)

In [83]:
def train_mlp_word2vec(x_train_pad, x_valid_pad, x_train_num, x_valid_num, y_train, y_valid):
    input_ts = Input(shape=(max_length,))
    input_exog = Input(shape=(x_train_num.shape[1], ))

    embedding_layer = Embedding(input_dim=num_words,
                                output_dim=embedding_dim,
                                weights=[embedding_matrix],
                                input_length=max_length,
                                trainable=False)(input_ts)

    pooled_output = Flatten()(embedding_layer)
    layer_1 = Dense(256, kernel_regularizer = tf.keras.regularizers.l2(0.01), activation = "relu")(pooled_output)
    dropout_1 = Dropout(0.2)(layer_1)
    layer_2 = Dense(128, activation = "relu")(dropout_1)

    layer_num1 = Dense(256, kernel_regularizer = tf.keras.regularizers.l2(0.01), activation = "relu")(input_exog)
    dropout_2 = Dropout(0.2)(layer_num1)
    layer_num2 = Dense(128, activation = "relu")(dropout_2)

    concat_ft = Concatenate()([layer_2, layer_num2])

    prediction = Dense(3, activation = "softmax")(concat_ft)
    model = Model(inputs=[input_ts, input_exog], outputs=prediction)
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-4 * math.exp(-0.1 * x))

    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

    model.compile(loss="categorical_crossentropy",
                    optimizer="adam",
                    metrics=["accuracy"])

    history = model.fit([x_train_pad, x_train_num], y_train,
                          epochs=20,
                          validation_data=([x_valid_pad, x_valid_num], y_valid),
                          batch_size=64,
                          verbose=1,
                          callbacks=[lr_scheduler, early_stopping])

    # Save the trained model
    model.save('mlp_word2vec.keras')

    return model, history

In [87]:
model, history = train_mlp_word2vec(x_train_pad, x_valid_pad, x_train_num, x_valid_num, y_train_cat, y_valid_cat)

Epoch 1/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 78s 200ms/step - accuracy: 0.3914 - loss: nan - val_accuracy: 0.6461 - val_loss: nan - learning_rate: 1.0000e-04
Epoch 2/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 74s 192ms/step - accuracy: 0.6731 - loss: nan - val_accuracy: 0.6650 - val_loss: nan - learning_rate: 9.0484e-05
Epoch 3/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 74s 192ms/step - accuracy: 0.7005 - loss: nan - val_accuracy: 0.6672 - val_loss: nan - learning_rate: 8.1873e-05
Epoch 4/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 75s 193ms/step - accuracy: 0.7223 - loss: nan - val_accuracy: 0.6642 - val_loss: nan - learning_rate: 7.4082e-05
Epoch 5/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 75s 193ms/step - accuracy: 0.7442 - loss: nan - val_accuracy: 0.6682 - val_loss: nan - learning_rate: 6.7032e-05
Epoch 6/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 74s 192ms/step - accuracy: 0.7622 - loss: nan - val_accuracy: 0.6686 - val_loss: nan - learning_rate: 6.0653e-05
Epoch 7/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 75s 193ms/step - accuracy: 0.7758 - loss: 

In [127]:
preds = model.predict([x_test_pad, x_test_num])
preds = np.argmax(preds, axis=1)
label = np.argmax(y_test_cat, axis=1)
print(classification_report(label, preds))

111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
              precision    recall  f1-score   support

           0       0.68      0.59      0.63      1001
           1       0.61      0.72      0.66      1430
           2       0.74      0.66      0.70      1103

    accuracy                           0.66      3534
   macro avg       0.68      0.66      0.66      3534
weighted avg       0.67      0.66      0.66      3534



In [128]:
def train_mlp_word2vec_tfidf(x_train_pad, x_valid_pad, x_train_num, x_valid_num, x_train_tfidf, x_valid_tfidf, y_train, y_valid):
    input_ts = Input(shape=(max_length,))
    input_exog = Input(shape=(x_train_num.shape[1], ))
    input_tfidf = Input(shape=(x_train_tfidf.shape[1], ))

    embedding_layer = Embedding(input_dim=num_words,
                                output_dim=embedding_dim,
                                weights=[embedding_matrix],
                                input_length=max_length,
                                trainable=False)(input_ts)

    pooled_output = Flatten()(embedding_layer)
    layer_1 = Dense(256, kernel_regularizer = tf.keras.regularizers.l2(0.01), activation = "relu")(pooled_output)
    dropout_1 = Dropout(0.2)(layer_1)
    layer_2 = Dense(128, activation = "relu")(dropout_1)

    layer_num1 = Dense(256, kernel_regularizer = tf.keras.regularizers.l2(0.01), activation = "relu")(input_exog)
    dropout_2 = Dropout(0.2)(layer_num1)
    layer_num2 = Dense(128, activation = "relu")(dropout_2)

    layer_tfidf1 = Dense(256, kernel_regularizer = tf.keras.regularizers.l2(0.01), activation = "relu")(input_tfidf)
    dropout_3 = Dropout(0.2)(layer_tfidf1)
    layer_tfidf2 = Dense(128, activation = "relu")(dropout_3)

    concat_ft = Concatenate()([layer_2, layer_num2, layer_tfidf2])

    final_dense = Dense(64, activation="relu")(concat_ft)
    prediction = Dense(3, activation = "softmax")(final_dense)
    model = Model(inputs=[input_ts, input_exog, input_tfidf], outputs=prediction)
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-4 * math.exp(-0.1 * x))

    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

    model.compile(loss="categorical_crossentropy",
                    optimizer="adam",
                    metrics=["accuracy"])

    history = model.fit([x_train_pad, x_train_num, x_train_tfidf], y_train,
                          epochs=20,
                          validation_data=([x_valid_pad, x_valid_num, x_valid_tfidf], y_valid),
                          batch_size=64,
                          verbose=1,
                          callbacks=[lr_scheduler, early_stopping])

    # Save the trained model
    model.save('mlp_word2vec_tfidf.keras')

    return model, history

In [158]:
vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train["cleaned_text"])
x_valid_tfidf = vectorizer.transform(x_valid["cleaned_text"])
x_test_tfidf = vectorizer.transform(x_test["cleaned_text"])
model, history = train_mlp_word2vec_tfidf(x_train_pad, x_valid_pad, x_train_num, x_valid_num, x_train_tfidf, x_valid_tfidf, y_train_cat, y_valid_cat)

Epoch 1/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 103s 262ms/step - accuracy: 0.4057 - loss: 4.7365 - val_accuracy: 0.6519 - val_loss: 1.0840 - learning_rate: 1.0000e-04
Epoch 2/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 101s 260ms/step - accuracy: 0.6513 - loss: 1.0833 - val_accuracy: 0.6832 - val_loss: 0.9577 - learning_rate: 9.0484e-05
Epoch 3/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 100s 258ms/step - accuracy: 0.7173 - loss: 0.8972 - val_accuracy: 0.7039 - val_loss: 0.9023 - learning_rate: 8.1873e-05
Epoch 4/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 101s 260ms/step - accuracy: 0.7596 - loss: 0.7899 - val_accuracy: 0.7013 - val_loss: 0.8786 - learning_rate: 7.4082e-05
Epoch 5/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 100s 259ms/step - accuracy: 0.7933 - loss: 0.7112 - val_accuracy: 0.7075 - val_loss: 0.8674 - learning_rate: 6.7032e-05
Epoch 6/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 100s 259ms/step - accuracy: 0.8187 - loss: 0.6472 - val_accuracy: 0.7053 - val_loss: 0.8938 - learning_rate: 6.0653e-05
Epoch 7/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 

In [159]:
preds = model.predict([x_test_pad, x_test_num, x_test_tfidf])
preds = np.argmax(preds, axis=1)
label = np.argmax(y_test_cat, axis=1)
print(classification_report(label, preds))

111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
              precision    recall  f1-score   support

           0       0.76      0.63      0.69      1001
           1       0.64      0.76      0.69      1430
           2       0.78      0.71      0.74      1103

    accuracy                           0.71      3534
   macro avg       0.73      0.70      0.71      3534
weighted avg       0.72      0.71      0.71      3534



# ROBERTA

In [8]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
roberta_model = TFRobertaModel.from_pretrained('roberta-base')
roberta_model.trainable = False

From d:\Egen\Kuliah\Discover NUS\Natural Language Processing\venv\Lib\site-packages\tf_keras\src\backend.py:873: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from 

In [ ]:
def get_roberta_embeddings(texts, max_length=128, batch_size=32):
    all_embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i : i + batch_size]
        
        tokens = tokenizer(batch_texts, padding="max_length", truncation=True,
                           max_length=max_length, return_tensors="tf")
        input_ids, attention_mask = tokens["input_ids"], tokens["attention_mask"]

        roberta_output = roberta_model(input_ids=input_ids, attention_mask=attention_mask)
        batch_embeddings = roberta_output.last_hidden_state[:, 0, :] 

        all_embeddings.append(batch_embeddings.numpy())
    
    return np.vstack(all_embeddings)

def train_mlp_roberta(x_train_embed, x_valid_embed, x_train_num, x_valid_num, y_train, y_valid):
    input_roberta = Input(shape=(x_train_embed.shape[1],))
    input_exog = Input(shape=(x_train_num.shape[1], ))

    layer_1 = Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_roberta)
    dropout_1 = Dropout(0.2)(layer_1)
    layer_2 = Dense(128, activation="relu")(dropout_1)

    layer_num1 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation="relu")(input_exog)
    dropout_2 = Dropout(0.2)(layer_num1)
    layer_num2 = Dense(128, activation="relu")(dropout_2)

    concat_ft = Concatenate()([layer_2, layer_num2])
    prediction = Dense(3, activation="softmax")(concat_ft)

    model = Model(inputs=[input_roberta, input_exog], outputs=prediction)

    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-4 * math.exp(-0.1 * x))
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

    history = model.fit([x_train_embed, x_train_num], y_train,
                        epochs=100,
                        validation_data=([x_valid_embed, x_valid_num], y_valid),
                        batch_size=64,
                        verbose=1,
                        callbacks=[lr_scheduler, early_stopping])

    model.save('mlp_roberta.keras')

    return model, history

In [ ]:
x_train_text = x_train["cleaned_text"].astype(str).tolist()
x_valid_text = x_valid["cleaned_text"].astype(str).tolist()
x_test_text = x_test["cleaned_text"].astype(str).tolist()
x_train_embed = get_roberta_embeddings(x_train_text)
x_valid_embed = get_roberta_embeddings(x_valid_text)
x_test_embed = get_roberta_embeddings(x_test_text)

In [25]:
model, history = train_mlp_roberta(x_train_embed, x_valid_embed, x_train_num, x_valid_num, y_train_cat, y_valid_cat)

Epoch 1/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3478 - loss: 4.8852 - val_accuracy: 0.4303 - val_loss: 2.0723 - learning_rate: 1.0000e-04
Epoch 2/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4000 - loss: 1.9797 - val_accuracy: 0.4969 - val_loss: 1.3230 - learning_rate: 9.0484e-05
Epoch 3/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4187 - loss: 1.3675 - val_accuracy: 0.4631 - val_loss: 1.1472 - learning_rate: 8.1873e-05
Epoch 4/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4578 - loss: 1.1916 - val_accuracy: 0.5013 - val_loss: 1.0753 - learning_rate: 7.4082e-05
Epoch 5/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4940 - loss: 1.1155 - val_accuracy: 0.5362 - val_loss: 1.0468 - learning_rate: 6.7032e-05
Epoch 6/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5262 - loss: 1.0623 - val_accuracy: 0.5995 - val_loss: 0.9901 - learning_rate: 6.0653e-05
Epoch 7/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - acc

In [28]:
preds = model.predict([x_test_embed, x_test_num])
preds = np.argmax(preds, axis=1)
label = np.argmax(y_test_cat, axis=1)
print(classification_report(label, preds))

111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.77      0.42      0.55      1001
           1       0.55      0.83      0.66      1430
           2       0.82      0.60      0.69      1103

    accuracy                           0.64      3534
   macro avg       0.71      0.62      0.63      3534
weighted avg       0.70      0.64      0.64      3534



In [40]:
def get_roberta_embeddings_3ndim(texts, max_length=128, batch_size=32):
    all_embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i : i + batch_size]
        
        tokens = tokenizer(batch_texts, padding="max_length", truncation=True,
                           max_length=max_length, return_tensors="tf")
        input_ids, attention_mask = tokens["input_ids"], tokens["attention_mask"]

        roberta_output = roberta_model(input_ids=input_ids, attention_mask=attention_mask)
        batch_embeddings = roberta_output.last_hidden_state

        all_embeddings.append(batch_embeddings.numpy())
    
    return np.vstack(all_embeddings)

def train_lstm_roberta(x_train_embed, x_valid_embed, x_train_num, x_valid_num, y_train, y_valid):
    input_roberta = Input(shape=(x_train_embed.shape[1], x_train_embed.shape[2]))
    input_exog = Input(shape=(x_train_num.shape[1], ))

    lstm_1 = LSTM(256, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_roberta)
    dropout_1 = Dropout(0.2)(lstm_1)
    lstm_2 = LSTM(128, return_sequences=False)(dropout_1)

    layer_num1 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation="relu")(input_exog)
    dropout_2 = Dropout(0.2)(layer_num1)
    layer_num2 = Dense(128, activation="relu")(dropout_2)

    concat_ft = Concatenate()([lstm_2, layer_num2])
    prediction = Dense(3, activation="softmax")(concat_ft)

    model = Model(inputs=[input_roberta, input_exog], outputs=prediction)

    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-4 * math.exp(-0.1 * x))
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

    history = model.fit([x_train_embed, x_train_num], y_train,
                        epochs=10,
                        validation_data=([x_valid_embed, x_valid_num], y_valid),
                        batch_size=64,
                        verbose=1,
                        callbacks=[lr_scheduler, early_stopping])

    model.save('lstm_roberta.keras')

    return model, history

In [30]:
x_train_embed_3ndim = get_roberta_embeddings_3ndim(x_train_text)
x_valid_embed_3ndim = get_roberta_embeddings_3ndim(x_valid_text)
x_test_embed_3ndim = get_roberta_embeddings_3ndim(x_test_text)

In [41]:
model, history = train_lstm_roberta(x_train_embed_3ndim, x_valid_embed_3ndim, x_train_num, x_valid_num, y_train_cat, y_valid_cat)

Epoch 1/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 172s 431ms/step - accuracy: 0.3905 - loss: 7.9416 - val_accuracy: 0.5828 - val_loss: 2.4863 - learning_rate: 1.0000e-04
Epoch 2/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 162s 415ms/step - accuracy: 0.5688 - loss: 2.2015 - val_accuracy: 0.6421 - val_loss: 1.3728 - learning_rate: 9.0484e-05
Epoch 3/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 160s 413ms/step - accuracy: 0.6249 - loss: 1.3776 - val_accuracy: 0.6104 - val_loss: 1.2424 - learning_rate: 8.1873e-05
Epoch 4/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 160s 414ms/step - accuracy: 0.6327 - loss: 1.1936 - val_accuracy: 0.6501 - val_loss: 1.0791 - learning_rate: 7.4082e-05
Epoch 5/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 157s 404ms/step - accuracy: 0.6564 - loss: 1.0762 - val_accuracy: 0.6719 - val_loss: 0.9970 - learning_rate: 6.7032e-05
Epoch 6/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 148s 382ms/step - accuracy: 0.6679 - loss: 1.0139 - val_accuracy: 0.6762 - val_loss: 0.9635 - learning_rate: 6.0653e-05
Epoch 7/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 

In [42]:
preds = model.predict([x_test_embed_3ndim, x_test_num])
preds = np.argmax(preds, axis=1)
label = np.argmax(y_test_cat, axis=1)
print(classification_report(label, preds))

111/111 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step
              precision    recall  f1-score   support

           0       0.73      0.54      0.62      1001
           1       0.60      0.77      0.67      1430
           2       0.77      0.67      0.71      1103

    accuracy                           0.67      3534
   macro avg       0.70      0.66      0.67      3534
weighted avg       0.69      0.67      0.67      3534



# Google-001

In [70]:
vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train["cleaned_text"])
x_valid_tfidf = vectorizer.transform(x_valid["cleaned_text"])
x_test_tfidf = vectorizer.transform(x_test["cleaned_text"])

In [71]:
df_embed = pd.read_csv("train_df_embed.csv")
g001_embed = df_embed["Embeddings"].values
x_001 = x.assign(g001_embed = g001_embed)
x_train, x_valid, y_train, y_valid = train_test_split(x_001, y, stratify=y, random_state=42, test_size=0.1, shuffle=True)
x_train_g001 = np.array([ast.literal_eval(item) for item in x_train["g001_embed"]])
x_valid_g001 = np.array([ast.literal_eval(item) for item in x_valid["g001_embed"]])

In [63]:
def train_mlp_g001(x_train_embed, x_valid_embed, x_train_num, x_valid_num, y_train, y_valid):
    input_g001 = Input(shape=(x_train_embed.shape[1],))
    input_exog = Input(shape=(x_train_num.shape[1], ))

    layer_1 = Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_g001)
    dropout_1 = Dropout(0.2)(layer_1)
    layer_2 = Dense(128, activation="relu")(dropout_1)

    layer_num1 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation="relu")(input_exog)
    dropout_2 = Dropout(0.2)(layer_num1)
    layer_num2 = Dense(128, activation="relu")(dropout_2)

    concat_ft = Concatenate()([layer_2, layer_num2])
    prediction = Dense(3, activation="softmax")(concat_ft)

    model = Model(inputs=[input_g001, input_exog], outputs=prediction)

    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-4 * math.exp(-0.1 * x))
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

    history = model.fit([x_train_embed, x_train_num], y_train,
                        epochs=100,
                        validation_data=([x_valid_embed, x_valid_num], y_valid),
                        batch_size=64,
                        verbose=1,
                        callbacks=[lr_scheduler, early_stopping])

    model.save('mlp_g001.keras')

    return model, history

In [66]:
model, history = train_mlp_g001(x_train_g001, x_valid_g001, x_train_num, x_valid_num, y_train_cat, y_valid_cat)

Epoch 1/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.3584 - loss: 5.1078 - val_accuracy: 0.4314 - val_loss: 2.0556 - learning_rate: 1.0000e-04
Epoch 2/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5058 - loss: 1.8358 - val_accuracy: 0.6839 - val_loss: 1.0947 - learning_rate: 9.0484e-05
Epoch 3/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6456 - loss: 1.1172 - val_accuracy: 0.6952 - val_loss: 0.8983 - learning_rate: 8.1873e-05
Epoch 4/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6694 - loss: 0.9313 - val_accuracy: 0.7166 - val_loss: 0.8070 - learning_rate: 7.4082e-05
Epoch 5/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6959 - loss: 0.8474 - val_accuracy: 0.7166 - val_loss: 0.7680 - learning_rate: 6.7032e-05
Epoch 6/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6976 - loss: 0.8098 - val_accuracy: 0.7246 - val_loss: 0.7436 - learning_rate: 6.0653e-05
Epoch 7/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - ac

In [72]:
def train_mlp_g001_tfidf(x_train_embed, x_valid_embed, x_train_num, x_valid_num, x_train_tfidf, x_valid_tfidf, y_train, y_valid):
    input_g001 = Input(shape=(x_train_embed.shape[1],))
    input_exog = Input(shape=(x_train_num.shape[1], ))
    input_tfidf = Input(shape=(x_train_tfidf.shape[1], ))

    layer_1 = Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_g001)
    dropout_1 = Dropout(0.2)(layer_1)
    layer_2 = Dense(128, activation="relu")(dropout_1)

    layer_num1 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation="relu")(input_exog)
    dropout_2 = Dropout(0.2)(layer_num1)
    layer_num2 = Dense(128, activation="relu")(dropout_2)

    layer_tfidf1 = Dense(256, activation="relu")(input_tfidf)
    dropout_3 = Dropout(0.2)(layer_tfidf1)
    layer_tfidf2 = Dense(128, activation="relu")(dropout_3)

    concat_ft = Concatenate()([layer_2, layer_num2, layer_tfidf2])
    last_dense = Dense(64, activation="relu")(concat_ft)
    prediction = Dense(3, activation="softmax")(last_dense)

    model = Model(inputs=[input_g001, input_exog, input_tfidf], outputs=prediction)

    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-4 * math.exp(-0.1 * x))
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

    history = model.fit([x_train_embed, x_train_num, x_train_tfidf], y_train,
                        epochs=100,
                        validation_data=([x_valid_embed, x_valid_num, x_valid_tfidf], y_valid),
                        batch_size=64,
                        verbose=1,
                        callbacks=[lr_scheduler, early_stopping])

    model.save('mlp_g001_tfidf.keras')

    return model, history

In [73]:
model, history = train_mlp_g001_tfidf(x_train_g001, x_valid_g001, x_train_num, x_valid_num, x_train_tfidf, x_valid_tfidf, y_train_cat, y_valid_cat)

Epoch 1/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 28s 60ms/step - accuracy: 0.3662 - loss: 4.4000 - val_accuracy: 0.6399 - val_loss: 1.8161 - learning_rate: 1.0000e-04
Epoch 2/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 22s 57ms/step - accuracy: 0.6365 - loss: 1.5237 - val_accuracy: 0.7359 - val_loss: 0.9144 - learning_rate: 9.0484e-05
Epoch 3/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 21s 54ms/step - accuracy: 0.7647 - loss: 0.8097 - val_accuracy: 0.7454 - val_loss: 0.7429 - learning_rate: 8.1873e-05
Epoch 4/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 21s 54ms/step - accuracy: 0.8184 - loss: 0.5914 - val_accuracy: 0.7290 - val_loss: 0.7234 - learning_rate: 7.4082e-05
Epoch 5/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 21s 55ms/step - accuracy: 0.8522 - loss: 0.4840 - val_accuracy: 0.7297 - val_loss: 0.7169 - learning_rate: 6.7032e-05
Epoch 6/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 22s 56ms/step - accuracy: 0.8690 - loss: 0.4284 - val_accuracy: 0.7254 - val_loss: 0.7371 - learning_rate: 6.0653e-05
Epoch 7/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 22s 5

# XLNET

In [76]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
xlnet_model = TFXLNetModel.from_pretrained("xlnet-base-cased")
xlnet_model.trainable = False

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


In [82]:
def get_xlnet_embeddings(texts, batch_size=32, max_length=128):
    all_embeddings = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Generating XLNet Embeddings"):
        batch_texts = texts[i : i + batch_size]
        tokens = tokenizer(batch_texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
        
        input_ids = tokens["input_ids"]
        attention_mask = tokens["attention_mask"]
        xlnet_output = xlnet_model(input_ids=input_ids, attention_mask=attention_mask)

        batch_embeddings = tf.reduce_mean(xlnet_output.last_hidden_state, axis=1)

        all_embeddings.append(batch_embeddings.numpy())

    return np.vstack(all_embeddings)

def train_mlp_xlnet(x_train_embed, x_valid_embed, x_train_num, x_valid_num, y_train, y_valid):
    input_xlnet = Input(shape=(x_train_embed.shape[1],))
    input_exog = Input(shape=(x_train_num.shape[1], ))

    layer_1 = Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_xlnet)
    dropout_1 = Dropout(0.2)(layer_1)
    layer_2 = Dense(128, activation="relu")(dropout_1)

    layer_num1 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation="relu")(input_exog)
    dropout_2 = Dropout(0.2)(layer_num1)
    layer_num2 = Dense(128, activation="relu")(dropout_2)

    concat_ft = Concatenate()([layer_2, layer_num2])
    prediction = Dense(3, activation="softmax")(concat_ft)

    model = Model(inputs=[input_xlnet, input_exog], outputs=prediction)

    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-4 * math.exp(-0.1 * x))
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

    history = model.fit([x_train_embed, x_train_num], y_train,
                        epochs=100,
                        validation_data=([x_valid_embed, x_valid_num], y_valid),
                        batch_size=64,
                        verbose=1,
                        callbacks=[lr_scheduler, early_stopping])

    model.save('mlp_xlnet.keras')

    return model, history

In [80]:
x_train_text = x_train["cleaned_text"].astype(str).tolist()
x_valid_text = x_valid["cleaned_text"].astype(str).tolist()
x_test_text = x_test["cleaned_text"].astype(str).tolist()
x_train_xlnet_embed = get_xlnet_embeddings(x_train_text)
x_valid_xlnet_embed = get_xlnet_embeddings(x_valid_text)
x_test_xlnet_embed = get_xlnet_embeddings(x_test_text)

Generating XLNet Embeddings: 100%|██████████| 111/111 [08:05<00:00,  4.37s/it]


In [83]:
model, history = train_mlp_xlnet(x_train_xlnet_embed, x_valid_xlnet_embed, x_train_num, x_valid_num, y_train_cat, y_valid_cat)

Epoch 1/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.3739 - loss: 5.3913 - val_accuracy: 0.5173 - val_loss: 3.7111 - learning_rate: 1.0000e-04
Epoch 2/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4589 - loss: 3.6228 - val_accuracy: 0.5438 - val_loss: 2.8463 - learning_rate: 9.0484e-05
Epoch 3/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5028 - loss: 2.7995 - val_accuracy: 0.5489 - val_loss: 2.3628 - learning_rate: 8.1873e-05
Epoch 4/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5327 - loss: 2.3210 - val_accuracy: 0.5879 - val_loss: 2.0350 - learning_rate: 7.4082e-05
Epoch 5/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5580 - loss: 2.0092 - val_accuracy: 0.5988 - val_loss: 1.8208 - learning_rate: 6.7032e-05
Epoch 6/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5649 - loss: 1.8074 - val_accuracy: 0.5973 - val_loss: 1.6686 - learning_rate: 6.0653e-05
Epoch 7/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - acc

In [85]:
preds = model.predict([x_test_xlnet_embed, x_test_num])
preds = np.argmax(preds, axis=1)
label = np.argmax(y_test_cat, axis=1)
print(classification_report(label, preds))

111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.69      0.36      0.47      1001
           1       0.54      0.78      0.64      1430
           2       0.71      0.61      0.65      1103

    accuracy                           0.61      3534
   macro avg       0.65      0.58      0.59      3534
weighted avg       0.63      0.61      0.60      3534



In [91]:
def train_lstm_xlnet(x_train_embed, x_valid_embed, x_train_num, x_valid_num, y_train, y_valid):
    input_xlnet = Input(shape=(x_train_embed.shape[1], x_train_embed.shape[2] ))
    input_exog = Input(shape=(x_train_num.shape[1], ))

    lstm_1 = LSTM(256, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_xlnet)
    dropout_1 = Dropout(0.2)(lstm_1)
    lstm_2 = LSTM(128, return_sequences=False)(dropout_1)

    layer_num1 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation="relu")(input_exog)
    dropout_2 = Dropout(0.2)(layer_num1)
    layer_num2 = Dense(128, activation="relu")(dropout_2)

    concat_ft = Concatenate()([lstm_2, layer_num2])
    last_dense = Dense(64, activation="relu")(concat_ft)
    prediction = Dense(3, activation="softmax")(last_dense)

    model = Model(inputs=[input_xlnet, input_exog], outputs=prediction)

    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-4 * math.exp(-0.1 * x))
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

    history = model.fit([x_train_embed, x_train_num], y_train,
                        epochs=100,
                        validation_data=([x_valid_embed, x_valid_num], y_valid),
                        batch_size=64,
                        verbose=1,
                        callbacks=[lr_scheduler, early_stopping])

    model.save('lstm_xlnet.keras')

    return model, history

In [92]:
model, history = train_lstm_xlnet(tf.expand_dims(x_train_xlnet_embed, axis=1), tf.expand_dims(x_valid_xlnet_embed, axis=1), x_train_num, x_valid_num, y_train_cat, y_valid_cat)

Epoch 1/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.3699 - loss: 7.4440 - val_accuracy: 0.4915 - val_loss: 2.2201 - learning_rate: 1.0000e-04
Epoch 2/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.4646 - loss: 1.8896 - val_accuracy: 0.5558 - val_loss: 1.2080 - learning_rate: 9.0484e-05
Epoch 3/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5411 - loss: 1.1753 - val_accuracy: 0.5908 - val_loss: 1.0572 - learning_rate: 8.1873e-05
Epoch 4/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5679 - loss: 1.0554 - val_accuracy: 0.5726 - val_loss: 1.0272 - learning_rate: 7.4082e-05
Epoch 5/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5789 - loss: 1.0075 - val_accuracy: 0.5602 - val_loss: 1.0328 - learning_rate: 6.7032e-05
Epoch 6/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5837 - loss: 0.9816 - val_accuracy: 0.5919 - val_loss: 0.9769 - learning_rate: 6.0653e-05
Epoch 7/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/ste

In [94]:
preds = model.predict([tf.expand_dims(x_test_xlnet_embed, axis = 1), x_test_num])
preds = np.argmax(preds, axis=1)
label = np.argmax(y_test_cat, axis=1)
print(classification_report(label, preds))

111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
              precision    recall  f1-score   support

           0       0.65      0.47      0.55      1001
           1       0.57      0.68      0.62      1430
           2       0.65      0.66      0.65      1103

    accuracy                           0.61      3534
   macro avg       0.62      0.60      0.61      3534
weighted avg       0.62      0.61      0.61      3534

